In [1]:
!uv pip install datasets pylate

Using Python 3.12.1 environment at: venv
Audited 2 packages in 119ms


In [ ]:
import argparse
import gzip
import json
import logging
import os
import pickle
import random
import tarfile
from datetime import datetime
from datasets import load_dataset, Dataset
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses,
)
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, SequentialEvaluator,TripletEvaluator
from sentence_transformers.training_args import BatchSamplers

from sentence_transformers import InputExample, LoggingHandler, SentenceTransformer, losses, util

#### Just some code to print debug information to stdout
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)
#### /print debug information to stdout

In [ ]:
!hf auth login

In [8]:
!uv pip install beir pylate ranx

Using Python 3.12.1 environment at: venv
Audited 3 packages in 29ms


In [9]:
from datasets import load_dataset

# Define the configuration for the nfcorpus-tr dataset
config = {
    "hf_corpus_path": "trmteb/fiqa-tr",
    "hf_corpus_split": "test",
}

# Load a small sample of the corpus split
sample_size = 10  # You can adjust this number
corpus_split = load_dataset(
    path=config["hf_corpus_path"],
    split=config["hf_corpus_split"],
)

# Print the features (columns) of the dataset
print("Dataset features:", corpus_split.features)

# Print the first few rows to inspect the data
print("\nFirst", sample_size, "rows of the dataset:")
for i in range(min(sample_size, len(corpus_split))):
    print(corpus_split[i])

Dataset features: {'query-id': Value('string'), 'corpus-id': Value('string'), 'score': Value('float64')}

First 10 rows of the dataset:
{'query-id': '8', 'corpus-id': '566392', 'score': 1.0}
{'query-id': '8', 'corpus-id': '65404', 'score': 1.0}
{'query-id': '15', 'corpus-id': '325273', 'score': 1.0}
{'query-id': '18', 'corpus-id': '88124', 'score': 1.0}
{'query-id': '26', 'corpus-id': '285255', 'score': 1.0}
{'query-id': '26', 'corpus-id': '350819', 'score': 1.0}
{'query-id': '34', 'corpus-id': '599545', 'score': 1.0}
{'query-id': '42', 'corpus-id': '272709', 'score': 1.0}
{'query-id': '42', 'corpus-id': '327263', 'score': 1.0}
{'query-id': '42', 'corpus-id': '331981', 'score': 1.0}


In [ ]:
from pylate import evaluation, indexes, models as pylate_models, retrieve

def evaluate_model(model_name: str, dataset_name: str) -> dict[str, float]:
    """
    Evaluates a ColBERT-based model on a Hugging Face information retrieval dataset.
    Works with datasets that contain multiple subsets (corpus, queries, default).

    Args:
        model_name: Hugging Face model id (e.g., "ozayezerceli/col-ettin-encoder-150M-TR")
        dataset_name: Dataset id (e.g., "trmteb/nfcorpus-tr")

    Returns:
        Dictionary of evaluation metrics.
    """


    corpus_split = load_dataset(dataset_name, name="corpus", split="corpus")
    queries_split = load_dataset(dataset_name, name="queries", split="queries")
    qrels_split = load_dataset(dataset_name, name="default", split="test")


    corpus_id_field, corpus_text_field = "_id", "text"
    query_id_field, query_text_field = "_id", "text"
    qrels_query_field, qrels_doc_field, qrels_score_field = "query-id", "corpus-id", "score"


    documents = [
        {"id": str(row[corpus_id_field]), "text": row[corpus_text_field]}
        for row in corpus_split
    ]


    queries = {str(row[query_id_field]): row[query_text_field] for row in queries_split}


    qrels = {}
    for row in qrels_split:
        qid, did = str(row[qrels_query_field]), str(row[qrels_doc_field])
        try:
            score = float(row[qrels_score_field])
        except (TypeError, ValueError):
            score = 1.0
        qrels.setdefault(qid, {})[did] = score

    
    model = pylate_models.ColBERT(model_name_or_path=model_name, document_length=300, query_length=None,trust_remote_code=True)
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)


    doc_emb = model.encode(
        [doc["text"] for doc in documents],
        batch_size=128,
        is_query=False,
        show_progress_bar=True,
    )
    index.add_documents([doc["id"] for doc in documents], doc_emb)


    q_emb = model.encode(
        list(queries.values()),
        batch_size=128,
        is_query=True,
        show_progress_bar=True,
    )


    scores = retriever.retrieve(q_emb, k=100)
    results = evaluation.evaluate(
        scores=scores,
        qrels=qrels,
        queries=list(queries.keys()),
        metrics=[
            f"ndcg@{k}" for k in [1, 3, 5, 10, 100]
        ]
        + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]
        + ["map", "recall@10", "recall@100", "precision@10", "precision@100"],
    )

    print(f"\n✅ Evaluation complete for model: {model_name}")
    return results


In [12]:
def compare_models(models: list[str], dataset_name: str) -> pd.DataFrame:
    """
    Evaluates multiple models on the same dataset and returns a comparison table.

    Args:
        models: List of Hugging Face model names.
        dataset_name: Dataset ID on Hugging Face.

    Returns:
        pandas DataFrame with all metrics per model.
    """
    results_list = []

    for model_name in models:
        metrics = evaluate_model(model_name, dataset_name)
        df = pd.DataFrame(metrics, index=[model_name])
        results_list.append(df)

    results_table = pd.concat(results_list)
    display(results_table.style.background_gradient(cmap="Blues").format(precision=4))
    return results_table


Fiqa Benchmark

In [21]:
models_to_test = [
    "ozayezerceli/col-ettin-encoder-150M-TR",
    "ozayezerceli/col-ettin-encoder-32M-TR",
    "ozayezerceli/ColmmBERT-base-TR",
    "ozayezerceli/ColmmBERT-small-TR",
    "ozayezerceli/colbert-femto-tr",
    "ozayezerceli/colbert-hash-pico-tr",
    "ozayezerceli/colbert-nano-tr",
    "ytu-ce-cosmos/turkish-colbert",
    "newmindai/TurkEmbed4Retrieval"
]

dataset_name = "trmteb/fiqa-tr"

comparison_df = compare_models(models_to_test, dataset_name)


2025-10-23 07:04:46 - Use pytorch device_name: cuda:0
2025-10-23 07:04:46 - Load pretrained ColBERT: ozayezerceli/col-ettin-encoder-150M-TR
2025-10-23 07:04:46 - No sentence-transformers model found with name ozayezerceli/col-ettin-encoder-150M-TR.
2025-10-23 07:04:48 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (768, 128).
2025-10-23 07:04:48 - Created a PyLate model from base encoder.
2025-10-23 07:04:48 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 52/52 [00:03<00:00, 16.38it/s]



✅ Evaluation complete for model: ozayezerceli/col-ettin-encoder-150M-TR
2025-10-23 07:15:22 - Use pytorch device_name: cuda:0
2025-10-23 07:15:22 - Load pretrained ColBERT: ozayezerceli/col-ettin-encoder-32M-TR
2025-10-23 07:15:22 - No sentence-transformers model found with name ozayezerceli/col-ettin-encoder-32M-TR.
2025-10-23 07:15:24 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (384, 128).
2025-10-23 07:15:24 - Created a PyLate model from base encoder.
2025-10-23 07:15:24 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 52/52 [00:01<00:00, 32.77it/s]



✅ Evaluation complete for model: ozayezerceli/col-ettin-encoder-32M-TR
2025-10-23 07:24:54 - Use pytorch device_name: cuda:0
2025-10-23 07:24:54 - Load pretrained ColBERT: ozayezerceli/ColmmBERT-base-TR
2025-10-23 07:24:59 - ✅ Index with FastPlaid backend.


...

Encoding queries (bs=128): 100%|██████████| 52/52 [00:03<00:00, 16.87it/s]



✅ Evaluation complete for model: ytu-ce-cosmos/turkish-colbert
2025-10-23 08:14:26 - Use pytorch device_name: cuda:0
2025-10-23 08:14:26 - Load pretrained ColBERT: newmindai/TurkEmbed4Retrieval
2025-10-23 08:14:32 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (768, 128).
2025-10-23 08:14:32 - Created a PyLate model from base encoder.
2025-10-23 08:14:32 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 52/52 [00:03<00:00, 14.11it/s]



✅ Evaluation complete for model: newmindai/TurkEmbed4Retrieval


,ndcg@1,ndcg@3,ndcg@5,ndcg@10,ndcg@100,hits@1,hits@3,hits@5,hits@10,hits@100,map,recall@10,recall@100,precision@10,precision@100
ozayezerceli/col-ettin-encoder-150M-TR,0.1235,0.1152,0.1227,0.1317,0.1731,0.1235,0.2238,0.2886,0.3627,0.7870,0.1039,0.1640,0.3250,0.0363,0.0079
ozayezerceli/col-ettin-encoder-32M-TR,0.1188,0.1095,0.1123,0.1221,0.1665,0.1188,0.2176,0.2654,0.3426,0.7716,0.0969,0.1512,0.3242,0.0343,0.0077
ozayezerceli/ColmmBERT-base-TR,0.2315,0.2085,0.2187,0.2438,0.3015,0.2315,0.4136,0.5185,0.7022,1.2948,0.1945,0.3089,0.5272,0.0702,0.0129
ozayezerceli/ColmmBERT-small-TR,0.1991,0.1863,0.1937,0.2126,0.2673,0.1991,0.3673,0.4522,0.6003,1.1466,0.1705,0.2687,0.4757,0.0600,0.0115
ozayezerceli/colbert-femto-tr,0.0139,0.0085,0.0093,0.0097,0.0184,0.0139,0.0154,0.0231,0.0293,0.1096,0.0077,0.0109,0.0464,0.0029,0.0011
ozayezerceli/colbert-hash-pico-tr,0.0710,0.0686,0.0695,0.0750,0.1093,0.0710,0.1389,0.1667,0.2114,0.5664,0.0592,0.0923,0.2285,0.0211,0.0057
ozayezerceli/colbert-nano-tr,0.0787,0.0685,0.0744,0.0850,0.1197,0.0787,0.1358,0.1836,0.2654,0.5988,0.0649,0.1106,0.2508,0.0265,0.0060
ytu-ce-cosmos/turkish-colbert,0.1373,0.1245,0.1287,0.1395,0.1917,0.1373,0.2469,0.3117,0.3981,0.9352,0.1126,0.1715,0.3805,0.0398,0.0094
newmindai/TurkEmbed4Retrieval,0.1142,0.1063,0.1162,0.1317,0.1816,0.1142,0.2176,0.2948,0.3981,0.9043,0.1015,0.1790,0.3780,0.0398,0.0090


In [ ]:
models_to_test = [
    "ozayezerceli/col-ettin-encoder-150M-TR",
    "ozayezerceli/col-ettin-encoder-32M-TR",
    "ozayezerceli/ColmmBERT-base-TR",
    "ozayezerceli/ColmmBERT-small-TR",
    "ozayezerceli/colbert-femto-tr",
    "ozayezerceli/colbert-hash-pico-tr",
    "ozayezerceli/colbert-nano-tr",
    "ytu-ce-cosmos/turkish-colbert",
    "newmindai/TurkEmbed4Retrieval"
]

dataset_name = "trmteb/scidocs-tr"

comparison_df = compare_models(models_to_test, dataset_name)

Generating test split: 100%|██████████| 29928/29928 [00:00<00:00, 2367543.00 examples/s]


2025-10-23 08:40:39 - Use pytorch device_name: cuda:0
2025-10-23 08:40:39 - Load pretrained ColBERT: ozayezerceli/col-ettin-encoder-150M-TR
2025-10-23 08:40:39 - No sentence-transformers model found with name ozayezerceli/col-ettin-encoder-150M-TR.
2025-10-23 08:40:42 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (768, 128).
2025-10-23 08:40:42 - Created a PyLate model from base encoder.
2025-10-23 08:40:42 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 8/8 [00:00<00:00, 16.05it/s]



✅ Evaluation complete for model: ozayezerceli/col-ettin-encoder-150M-TR
2025-10-23 08:44:05 - Use pytorch device_name: cuda:0
2025-10-23 08:44:05 - Load pretrained ColBERT: ozayezerceli/col-ettin-encoder-32M-TR
2025-10-23 08:44:06 - No sentence-transformers model found with name ozayezerceli/col-ettin-encoder-32M-TR.
2025-10-23 08:44:07 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (384, 128).
2025-10-23 08:44:07 - Created a PyLate model from base encoder.
2025-10-23 08:44:07 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 8/8 [00:00<00:00, 34.70it/s]


...


✅ Evaluation complete for model: ytu-ce-cosmos/turkish-colbert
2025-10-23 08:59:33 - Use pytorch device_name: cuda:0
2025-10-23 08:59:33 - Load pretrained ColBERT: newmindai/TurkEmbed4Retrieval
2025-10-23 08:59:41 - The checkpoint does not contain a linear projection layer. Adding one with output dimensions (768, 128).
2025-10-23 08:59:41 - Created a PyLate model from base encoder.
2025-10-23 08:59:41 - ✅ Index with FastPlaid backend.


Encoding queries (bs=128): 100%|██████████| 8/8 [00:00<00:00, 13.65it/s]



✅ Evaluation complete for model: newmindai/TurkEmbed4Retrieval


,ndcg@1,ndcg@3,ndcg@5,ndcg@10,ndcg@100,hits@1,hits@3,hits@5,hits@10,hits@100,map,recall@10,recall@100,precision@10,precision@100
ozayezerceli/col-ettin-encoder-150M-TR,0.0830,0.0671,0.0592,0.0697,0.1090,0.0830,0.1880,0.2600,0.3540,0.9290,0.0448,0.0722,0.1894,0.0354,0.0093
ozayezerceli/col-ettin-encoder-32M-TR,0.0780,0.0606,0.0542,0.0648,0.1014,0.0780,0.1670,0.2390,0.3350,0.8670,0.0409,0.0680,0.1764,0.0335,0.0087
ozayezerceli/ColmmBERT-base-TR,0.1250,0.1018,0.0854,0.1020,0.1556,0.1250,0.2810,0.3610,0.5130,1.3020,0.0680,0.1039,0.2642,0.0513,0.0130
ozayezerceli/ColmmBERT-small-TR,0.1120,0.0915,0.0781,0.0942,0.1424,0.1120,0.2570,0.3380,0.4840,1.2040,0.0615,0.0980,0.2444,0.0484,0.0120
ozayezerceli/colbert-femto-tr,0.0220,0.0166,0.0156,0.0190,0.0377,0.0220,0.0470,0.0720,0.1030,0.3810,0.0117,0.0209,0.0778,0.0103,0.0038
ozayezerceli/colbert-hash-pico-tr,0.0640,0.0483,0.0425,0.0523,0.0839,0.0640,0.1310,0.1820,0.2720,0.7390,0.0321,0.0552,0.1501,0.0272,0.0074
ozayezerceli/colbert-nano-tr,0.0750,0.0525,0.0472,0.0576,0.0916,0.0750,0.1400,0.2040,0.2990,0.8060,0.0356,0.0608,0.1639,0.0299,0.0081
ytu-ce-cosmos/turkish-colbert,0.0530,0.0386,0.0338,0.0410,0.0659,0.0530,0.1040,0.1440,0.2080,0.5740,0.0278,0.0423,0.1172,0.0208,0.0057
newmindai/TurkEmbed4Retrieval,0.0770,0.0653,0.0583,0.0733,0.1210,0.0770,0.1850,0.2610,0.3970,1.0900,0.0479,0.0809,0.2218,0.0397,0.0109


: 